In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("data_cleaned_1.xlsx")

# Chuẩn hoá dữ liệu

> Làm gọn tên sản phẩm

Đồng hồ thông minh

In [3]:
mask_smartwatch = df['category'].str.contains('đồng hồ thông minh', case=False, na=False)

In [4]:
name_map = {
    ' Đồng Hồ Thông Minh Trẻ em AMA Q12S Model mới nhất ra mắt tháng 12/2021 - Hàng nhập khẩu': 'AMA Q12S',
    'Đồng hồ Thông minh Trẻ em AMA Watch Q15 lắp SIM nghe gọi 2 chiều 3 màu Xanh Hồng Đen Hàng chinh hãng': 'AMA Watch Q15',
    'Đồng hồ Thông minh Mini có Rung Gắn sim Định vị GPS, Wifi Đo huyết áp Nhịp tim nhắc nhở Vận động gọi Video call AMA Watch FA91S - Hàng nhập khẩu': 'AMA Watch FA91S',
    'Đồng hồ Thông minh Trẻ em Học sinh Tiểu học THCS THPT Chống nước Lắp sim Định vị Kép Smart Watch AMA HW13 Hàng nhập khẩu': 'AMA HW13',
    'Đồng hồ Thông minh Lắp Sim Định vị GPS Google kết nối Wifi 4G Blueltooth xem được Video Youtube TikTok tải App FB Messenger Skype WhatsApp Viber Wechat Kakao Line Tele chơi Game AMA Smart watch TK Android 8.1 cho Trẻ em Người lớn Hàng nhập khẩu': 'AMA TK Watch Android 8.1',
    'Apple Watch SE 2024 GPS (Viền nhôm, dây cao su)': 'Apple Watch SE 2024 viền nhôm dây cao su',
    'Vòng Đeo Tay Thông Minh Xiaomi Smart Band 9 Active – Hàng Chính Hãng': 'Xiaomi Smart Band 9 Active',
    'Garmin Forerunner 570 47mm dây silicone': 'Garmin Forerunner 570 47mm',
    'Amazfit Active 2 Square Sapphire 43.3mm dây daMẫu mới': 'Amazfit Active 2 Square 43.3mm dây da',
    'Garmin Forerunner 570 42.4mm dây silicone': 'Garmin Forerunner 570 42.4mm',
    'Đồng Hồ Thông Minh Xiaomi Redmi Watch 5 ACTIVE – Hàng Chính Hãng': 'Xiaomi Redmi Watch 5 Active',
    'Vòng Đeo Tay Thông Minh Xiaomi Mi Band 10 - Bản Quốc Tế - Hàng Chính Hãng': 'Xiaomi Mi Band 10',
    'Apple Watch Series 10 GPS (Viền nhôm, dây cao su)': 'Apple Watch Series 10 viền nhôm dây cao su',
    'Samsung Galaxy Watch Ultra LTE 47mm (2025) dây silicone': 'Samsung Galaxy Watch Ultra 47mm dây silicone',
    'Đồng Hồ Thông Minh Trẻ Em MAI DUY Y85S, Lắp Sim Nghe Gọi - Định Vị Cảnh Báo Nguy Hiểm, Chống Nước  -Hàng Nhập Khẩu': 'MAI DUY Y85S',
    'Đồng Hồ Thông Minh Trẻ Em AMA Watch Q12B Hàng nhập khẩu': 'AMA Watch Q12B',
    'Huawei Watch 5 46mm dây cao su': 'Huawei Watch 5 46mm dây cao su',
    'Samsung Galaxy Watch8 LTE 40mm dây siliconeMẫu mới': 'Samsung Galaxy Watch8 40mm dây silicone',
    'Huawei Watch 5 46mm dây compositeMẫu mới': 'Huawei Watch 5 46mm dây composite',
    'Garmin Venu X1 Sapphire 51.2mm dây nylonMẫu mới': 'Garmin Venu X1 51.2mm dây nylon',
    'Garmin Forerunner 570 47mm dây silicone': 'Garmin Forerunner 570 47mm dây silicone',
    'Amazfit Active 2 Square Sapphire 43.3mm dây daMẫu mới': 'Amazfit Active 2 Square 43.3mm dây da',
    'Garmin Forerunner 570 42.4mm dây silicone': 'Garmin Forerunner 570 42.4mm',
    'Samsung Galaxy Watch Ultra LTE 47mm (2025) dây silicone': 'Samsung Galaxy Watch Ultra 47mm dây silicone',
    'Samsung Galaxy Watch8 LTE 40mm dây siliconeMẫu mới': 'Samsung Galaxy Watch8 40mm dây silicone',
    'Đồng hồ Thông minh gắn Sim nghe gọi Chống nước IP67 Gọi Video call Định vị GPS, WIfi dàn cho Trẻ em học sinh cấp 1 2 5 6 7 8 9 10 11 12 tuổi AMA Watch D32 Hàng nhập khẩu': 'AMA Watch D32',
    'Đồng Hồ Định Vị Trẻ Em Wonlex KT18 - Hàng Chính Hãng': 'Wonlex KT18',
    'Đồng hồ thông minh Samsung Galaxy Watch5 40mm SM-R900N - Hàng chính hãng': 'Samsung Galaxy Watch5 SM-R900N 40mm',
    'Đồng hồ Thông minh Chống nước IP67 Dành cho Trẻ em nhuẩn AMA Watch Q16S Hàng chính hãng': 'AMA Watch Q16S',
    'Đồng Hồ Thông Minh cho Trẻ em 6 tuổi, 7 tuổi, 8 tuổi, 9 tuổi, 10 tuổi, 11 tuổi Lắp Sim Độc lập nghe gọi 2 chiều AMA Watch Y92 Chống nước Định vị Wifi độ chính xác cao Hàng nhập khẩu': 'AMA Watch Y92',
    'Đồng Hồ Thông Minh AMA Watch Nam Nữ chống nước | Gọi điện thoại Xem tin nhắn Theo dõi Sức khỏe vận động Thiết kế nhỏ gọn Thời trang Hàng nhập khẩu': 'AMA Watch D06',
    'Đồng Hồ Thông Minh Nữ AMA Watch S27 Thời trang Kết nối Bluetooth Theo dõi Sức khỏe Vận động Màn hình tròn Hàng chính hãng': 'AMA Watch S27'
}

# Tạo cột mới 'products' bằng cách ánh xạ product_name sang name_map
df.loc[mask_smartwatch, 'product_name'] = df.loc[mask_smartwatch, 'product_name'].map(name_map)

In [5]:
df_watch = df[mask_smartwatch].copy()

In [6]:
# Danh sách thương hiệu phổ biến
brands = ['AMA', 'Xiaomi', 'Apple', 'Samsung', 'Garmin', 'Huawei', 'Amazfit', 'Wonlex', 'MAI DUY']

# Hàm tách brand và model, xử lý riêng với AMA
def split_brand_model(product):
    for brand in brands:
        if product.startswith(brand):
            model = product[len(brand):].strip()
            if brand == "AMA" and model.startswith("Watch"):
                model = model[len("Watch"):].strip()
            return pd.Series([brand, model])
    return pd.Series(['Unknown', product])

# Áp dụng
df_watch[['brand', 'model']] = df_watch['product_name'].apply(split_brand_model)

# Gán ngược vào df chính
df.loc[mask_smartwatch, 'brand'] = df_watch['brand'].values
df.loc[mask_smartwatch, 'model'] = df_watch['model'].values

In [7]:
import re

In [8]:
info_patterns = r'\d+\.?\d*mm|viền [a-zA-ZÀ-ỹ\s]+|dây [a-zA-ZÀ-ỹ\s]+'

def split_model_info(text):
    # Tìm các phần info
    infos = re.findall(info_patterns, text, flags=re.IGNORECASE)
    info_str = ' '.join(infos).strip()

    # Loại bỏ info khỏi model
    model_cleaned = text
    for i in infos:
        model_cleaned = model_cleaned.replace(i, '')

    # Xoá khoảng trắng thừa
    model_cleaned = re.sub(r'\s{2,}', ' ', model_cleaned).strip()

    return pd.Series([model_cleaned, info_str])

In [9]:
df[['model', 'info']] = df['model'].fillna('').apply(split_model_info)

# Lưu file

In [10]:
df.to_excel("data_cleaned_2.xlsx", index=False)